In [ ]:
import pandas as pd

# Display all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

all_df=pd.read_csv("./8月召回任务销售拜访记录 8.13~8.20 共3931条.csv")

all_df.head(2)

In [14]:
# Import the base64 encoding library.
import base64, os, time
import logging

# Configure the logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

proxy_object = {"http": "http://127.0.0.1:8001", "https": "http://127.0.0.1:8001"}


from openai import AzureOpenAI

client_gpt4o = AzureOpenAI(
    api_version="2024-03-01-preview",
    azure_endpoint="https://xm-ai-us2.openai.azure.com",
    api_key=os.getenv("AZURE_GPT4O_API_KEY", ""),
)

client_gpt4o_mini = AzureOpenAI(
    api_version="2024-03-01-preview",
    azure_endpoint="https://xm-ai-us.openai.azure.com",
    api_key=os.getenv("AZURE_GPT4O_MINI_API_KEY", ""),
)


def call_azure_openai(messages=[], retrying=1, is_gpt4o=False) -> (str, bool):
    if retrying < 0:
        return "超过了最大重试次数", False
    completion = None
    ## gpt3.5:  gpt-35-turbo-16k,
    ## got4o:   gpt-4o
    ## got4o-mini:   gpt-4o-mini
    model = "gpt-4o-mini"
    client_to_use = client_gpt4o_mini
    if is_gpt4o:
        logging.info(f"using GPT-4o...:{messages}")
        model = "gpt-4o"
        client_to_use = client_gpt4o
    try:
        completion = client_to_use.chat.completions.create(
            model=model,
            temperature=0.1,
            max_tokens=4095,
            messages=messages,
            response_format={"type": "json_object"},
        )
        response = completion.choices[0].message.content
        if (
            len(completion.choices) <= 0
            or f"{completion.choices[0].finish_reason}" == "content_filter"
        ):
            return f"azure过滤了本次请求:{completion.choices[0].to_dict()}", False
        if response is None:
            logging.info(f"azure API返回了异常:{completion.to_dict()}")
            time.sleep(10)
            return call_azure_openai(
                messages=messages,
                retrying=retrying - 1,
                is_gpt4o=is_gpt4o,
            )
        logging.info(f"total usage:{completion.usage}")
        return response, True
    except Exception as e:
        logging.info(
            f"请求azure接口报错了:{e}\n messages:{messages}, completion:{completion}"
        )
        if retrying <= 0 or "Error code: 400" in f"{e}":
            return f"{e}", False
        logging.info(f"重试中...{retrying}, messages:{messages}")
        return call_azure_openai(
            messages=messages,
            retrying=retrying - 1,
            is_gpt4o=is_gpt4o,
        )


def call_ai_api_to_get_insigns(visit_text=""):
    result = {}
    json_text, is_ok = call_azure_openai(
        is_gpt4o=False,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": """
用户会给发给你一系列销售员对客户的拜访记录，请你用JSON回答以下6个问题：
1. 客户长时间未下单原因？
2. 本次拜访前销售做了什么准备？
3. 销售向客户推荐了平台的什么活动？（入未推荐则回答‘无’）
4. 销售向客户推荐了哪些具体的商品？（入未推荐则回答‘无’）
5. 客户主要采买渠道？
6. 客户不愿意拜访期间下单的原因是？

**请注意，‘安佳’，‘铁塔’一般来说是商品名字，而不太可能是活动名字，活动名字一般带有‘专享’、‘清仓’、‘特价’、‘活动’等字样**
**请用问题标题来做JSON的key，比如{"客户长时间未下单原因":"其他原因-近期已下单"}**
**请注意，以上问题中，“客户不愿意拜访期间下单的原因”和“客户长时间未下单原因”请你使用以下结构化的选项回答**：

- 价格：[价格贵、价格问题（未具体说明）]
- 用量少或不需要：[用量小或季节性减少、不需要、生意不好]
- 品质：[品质不稳定、品质差、其他品质问题]
- 有其他供应商或渠道：[有固定或其他供应商、其他平台更便宜、其他供应商品质好、其他供应商更方便、其他供应商给账期、其他供应商促销]
- 已倒闭或店铺关闭：[倒闭或不再营业、装修或暂停营业、搬迁]
- 其他原因：[近期已下单、客户个人休假、客户转其他平台或改变产品线、未明确原因]
- 配送：[配送不方便、配送不及时、配送门槛高、配送费问题]
- 售后：[售后服务不好、售后麻烦、可售后时间太短、售后不及时、其他售后问题]
- 公司管控或政策原因：[总部采购政策变动、总部不允许外采]
- 规格：[规格太大或太小、其他规格问题]
""",
                    }
                ],
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": visit_text}],
            },
        ],
    )

    logging.info(f"json_text:{json_text}, visit_text:{visit_text}")
    return json_text


# top2_df = all_df.head(2).copy()
all_df["AI分析"] = all_df["跟进情况描述"].apply(call_ai_api_to_get_insigns)
all_df.to_csv("./8月召回任务销售拜访记录8.13~8.20共3931条_GPT4o_mini.csv")
all_df.head(10)[["客户名称", "AI分析", "跟进情况描述"]]

2024-08-22 18:24:08 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-08-22 18:24:08 - DEBUG - load_verify_locations cafile='/Users/tangpeng/Documents/github/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-08-22 18:24:09 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-08-22 18:24:09 - DEBUG - load_verify_locations cafile='/Users/tangpeng/Documents/github/.venv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-08-22 18:24:09 - DEBUG - Request options: {'method': 'post', 'url': '/deployments/gpt-4o-mini/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '\n用户会给发给你一系列销售员对客户的拜访记录，请你用JSON回答以下6个问题：\n1. 客户长时间未下单原因？\n2. 本次拜访前销售做了什么准备？\n3. 销售向客户推荐了平台的什么活动？（入未推荐则回答‘无’）\n4. 销售向客户推荐了哪些具体的商品？（入未推荐则回答‘无’）\n5. 客户主要采买渠道？\n6. 客户不愿意拜访期间下单的原因是？\n\n**请注意，‘安佳’，‘铁塔’一般来说是商品名字，而不太可能是活动名字，活动名字一般带有‘专享’、‘清仓’、‘特价’、‘活动’等字样**\n**请用问题标题来做JSON的key，比如{"客户长时间未下单原因":"其他原因-近期已下